# ResNet

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline

See MIT License in https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 Online.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

import os
import pathlib
import time
from datetime import datetime

np.random.seed(813306)

# User inputs ##

flist = ['private_balanced'] # List dataset directory names. WormsTwoClass Lightning2 Earthquakes GunPoint 
batch_size = 64 # Set to -1 to use Wang et al settings
nb_epochs = 1500 # Wang: 1500
k = 10 # For k-fold cross validation. If k=1, the original test-train split is used.
m = 1 # Number of repetitions of k-fold cross validation (if k>1).
k_fold_seed = 87
tensorboard = True # Set to True to write logs for use by TensorBoard

# Directories
logs_dir = '../logs'
tensorboard_dir = '../logs/tensorboard'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now())
logs_dir = logs_dir +'/' + timestamp
tensorboard_dir = tensorboard_dir +'/' + timestamp

# Input directory
if 'private' in flist[0]:
    fdir = '../data/private_data/private_events_dev2' 
else:
    fdir = '../data' 

# Results

### GunPoint 2018-12-22T14:40

Batch size = 64

|Run |Loss |Accuracy | Epoch index     | Duration
|:---|:--- |:---     |:----------      |:-------------
|0  |1.2543987577373627e-05  |0.9799999952316284 |1494 |3mins  |
|1  |0.00013335476629436016  |0.9699999952316284 |1496 |3mins  |


In [ ]:
def plot_confusion_matrix(cm, title='Normalised confusion matrix', save=False):
    ''' Plot the normalised confusion matrix
    Parameters
    cm : array - normalised confusion matrix
    Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
    'Confusion Matrix' https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
    '''
    classes = ['Positive', 'Negative']
    cmap=plt.cm.Blues
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.clim(0, 1)
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.tight_layout()
    if save:
        plt.savefig('cm_resnet.png', bbox_inches='tight')

In [ ]:
def plot_roc(y_true, y_probs, save=False): 
    ''' Plot ROC and return AUC
    Parameters
    y_true : vector of true class labels.
    y_probs : array of predicted probabilities, one column for each class.
    Returns
    auc : float
    '''
    fpr, tpr, thresholds = roc_curve(y_true, y_probs[:,1])
    auc = roc_auc_score(y_true, y_probs[:,1])
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.show()
    if save:
        plt.savefig('roc_resnet.png', bbox_inches='tight')
    return auc

# Build and train ResNet

In [ ]:
def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling2D()(y)   
    out = Dense(nb_classes, activation='softmax')(full)
    print ('        -- model was built.')
    return x, out
 
       
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
   
    
def train_model(fname, x_train, y_train, x_test, y_test, label="0"):
    
    print('Running dataset', fname)
    if batch_size == -1:
        batch = int(min(x_train.shape[0]/10, 16)) # Wang et al. setting.
    else:
        batch=batch_size
        
    nb_classes = len(np.unique(y_test))
     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
     
     
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
      
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))
     
     
    x , y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = Model(x, y)
    #print(model.summary())
    
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    pathlib.Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    callbacks = [reduce_lr]
    if tensorboard:
        tb_dir = tensorboard_dir+'/'+fname+'_'+label
        pathlib.Path(tb_dir).mkdir(parents=True, exist_ok=True) 
        print('Tensorboard logs in', tb_dir)
        callbacks.append(keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0))
  
    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
    end = time.time()
    log = pd.DataFrame(hist.history)  

    # Print and save results. Print the testing results which has the lowest training loss.
    print('Training complete on', fname)
    duration_minutes = str(round((end-start)/60))
    print('Training time ', end-start, 'seconds, which is about', duration_minutes, 'minutes.')    
    print('Selected the test result with the lowest training loss. Loss and validation accuracy are -')
    idx = log['loss'].idxmin()
    loss = log.loc[idx]['loss']
    val_acc = log.loc[idx]['val_acc']
    print(loss, val_acc, 'at index', str(idx), ' (epoch ', str(idx+1), ')')
    summary = '|' + label + '  |'+str(loss)+'  |'+str(val_acc)+' |'+str(idx)+' |'+ duration_minutes + 'mins  |'
    summary_csv = label+','+str(loss)+','+str(val_acc)+','+str(idx)+','+ duration_minutes 
    
    # Save summary file and log file.
    print('Tensorboard logs in', tb_dir)
    with open(logs_dir+'/'+fname+'/resnet_summary.csv', 'a+') as f:
        f.write(summary_csv)
        f.write('\n')
        print('Added summary row to ', logs_dir+'/'+fname+'/resnet_summary.csv')  
    print('Saving logs to',logs_dir+'/'+fname+'/history_'+label+'.csv')
    log.to_csv(logs_dir+'/'+fname+'/history_'+label+'.csv')
    
    return summary, model


# main
results = []
for each in flist:
    fname = each
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    # k-fold cross validation setup
    if k > 1:
        x_all = np.concatenate((x_train, x_test), axis=0)
        y_all = np.concatenate((y_train, y_test), axis=0)
        kfold = RepeatedStratifiedKFold(n_splits=k, n_repeats=m, random_state=k_fold_seed)
        count = 0
        for train, test in kfold.split(x_all):
            x_train, y_train, x_test, y_test = x_all[train], y_all[train], x_all[test], y_all[test]
            summary, model = train_model(fname, x_train, y_train, x_test, y_test, str(count))
            results.append(summary)
            count = count + 1
    else:
        summary, model = train_model(fname, x_train, y_train, x_test, y_test)
        results.append(summary)
        
print('DONE')
print(fname, timestamp)
print('train:test', y_train.shape[0], y_test.shape[0])
for each in results:
    print(each)


In [ ]:
# Done at
'{:%Y-%m-%dT%H:%M}'.format(datetime.now())

# Metrics

In [ ]:
# Use trained model (after all epochs) to make predictions
do_print = True
x_input = x_test
y_input = y_test
y_input = y_input - y_input.min()
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_input = (x_input - x_train_mean)/(x_train_std)
x_input = x_input.reshape(x_input.shape + (1,1,))
nb_classes = len(np.unique(y_input))
y_input = (y_input - y_input.min())/(y_input.max()-y_input.min())*(nb_classes-1)
# Class balance
n0 = (y_input == 0).sum()
n1 = (y_input == 1).sum()
# Calculate model prediction
y_probs = model.predict_on_batch(x_input)
y_class = y_probs.argmax(axis=1)
cm = confusion_matrix(y_input, y_probs.argmax(axis=1), labels=[1,0])
acc_calc = (cm[0][0]+cm[1][1])/(cm.sum())
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
if do_print:
    print('Predicted class probabilities:\n', y_probs[:5,:])
    print('Pred', y_class[:20])
    print('True', y_input[:20].astype(int))
    print(cm)
    print('Calculated accuracy:',acc_calc)
    print('Class balance in test set:', n0, 'to', n1, 'i.e.', n0/(n0+n1))
    print('Normalised confusion matrix:\n', cm_norm)
title = 'Normalised confusion matrix'
plot_confusion_matrix(cm_norm, title=title, save=True)

# ROC and AUC
auc = plot_roc(y_input, y_probs)
print('AUC:', auc)

In [ ]:
file = logs_dir+'/'+fname+'/resnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
accuracy = data.iloc[:,2]
print('Results from file', file)
print(data.describe())
print('Accuracy mean and 95% confidence level is', accuracy.mean(), accuracy.std()*1.96)
print('95% confidence interval is', accuracy.quantile(0.0025), 'to', accuracy.quantile(0.975))
plt.figure(0)
data.boxplot(column='loss')
plt.figure(1)
data.boxplot(column='val_acc')
data.hist(column='val_acc')
print('Rows with val_acc<1 are')
data[data.val_acc<0.99]

# Summary results

95% confidence :-

|Timestamp |Model |Mean val_acc | +/- | lower |  upper  | Comment
|:---|:--- |:---  |:---    |:----------      |:------------- |:-------------
|2018-12-01T08:43  |MLP  |0.9715 | 0.0727 |0.8500 |1.0  | 10-fold, 10 resamples |
|2018-12-22T15:54  |ResNet  |0.9960 | 0.0301 | 0.9124 |1.0  | 10-fold, 10 resamples |

# Work in progress - results for ACI dataset

Data samples truncated to 12,000 datapoints.
Using the 'small' dataset - Train on 164 samples, validate on 1484 samples. Batch size 64 was too big for GPU. Batch size 32 ran, with 100% GPU utilisation.


|Timestamp         |Model  |epochs  |loss   | val_acc | at epoch  | run time | Comment
|:------           |:---   |:---    |:----  |:-----   |:-----     |:-----    |:-------------
|2019-02-03T15:30  |ResNet | 50     |0.5681 | 0.7305  |18         | 19mins   |164 train, 1484 test |
|2019-02-03T16:36  |ResNet | 50     |0.5391 | 0.6923  |46         |  5mins   |154 train, 52 test   |
|2019-02-03T16:46  |ResNet | 50     |0.5890 | 0.7208  |26         |  3mins   |52 train,  154 test   |
|2019-02-03T17:01  |ResNet | 50     |0.5883 | 0.7208  |26         |  3mins   | "  batch size 16   |
|2019-02-03T17:08  |ResNet | 50     |0.0308 | 0.5649 |1490        | 92mins   | "  acc=1.0   |

## Event windowing applied to reduce data to 1000 datapoints
Batch size 32. Train on 50 samples, validate on 150 samples.

|Timestamp         |Dataset |Model  |epochs  |loss   | val_acc | at epoch  | run time | Comment
|:------           |:---    |:---   |:---    |:----  |:-----   |:-----     |:-----    |:-------------
|2019-02-17T08:52  |mini_dog2 |ResNet | 1500   |0.007090 |0.5000 |1468 |9mins  | acc=1.0
|2019-02-17T09:08  |mini      |ResNet |1500   |0.03282  |0.5066 |1493 |9mins  | acc = 0.98
|2019-02-17T17:49 |mini_dog2 but 150:50 train:test |ResNet | 1500 |0.00125 | 0.5000 | 1467 |15mins  |


## Removed samples where no event occured

|Timestamp         |Dataset |Model  |epochs  |loss   | val_acc | at epoch  | run time | Comment
|:------           |:---    |:---   |:---    |:----  |:-----   |:-----     |:-----    |:-------------
|2019-02-22T18:02  |private_dog0 (135:45)|MLP | 1500   |0.000256 |0.489 |1473 |1mins  | acc=1.0
|2019-02-22T18:05  |private_dog1 (111:37)|MLP | 1500   |2.6e-6 |0.459 |1190 |1mins  | acc=1.0
|2019-02-23T11:14  |private_dog2 (120:40)|MLP | 1500   |9e-6 |0.525 |1414 |1mins  | acc=1.0
|2019-02-23T08:12  |private_dog0 (135:45)|ResNet | 1500   |0.0802 |0.667 |1499 |13mins  | acc=0.99
|2019-02-23T08:59  |private_dog1 (111:37)|ResNet | 1500   |0.000876 |0.568 |1480 |10mins  | acc=1.0
|2019-02-21T16:48  |private_dog2 (120:40)|ResNet | 1500   |0.000393 |0.625 |1423 |12mins  | acc=1.0
|2019-02-21T17:08 |private_events_dev (831:278)|ResNet | 1500   |8.128e-05|0.637 |1468 |75mins  | acc=1.0, batch size 64
|2019-02-22T07:25  |private_mini (50:150) |MLP |1500 | 1.8502e-06 | 0.627 | 1232 | 1min | acc=1.0, batch size 32
|2019-02-22T08:41  |private_events_dev (831:278) |MLP  |1500 | 0.0182 | 0.6331 | 1416 | 3mins | acc=0.98 batch size 64



NB private_events_dev_TEST is unbalanced; with 0.676 in class 0


## Cross fold validation

2019-02-22T08:52/private_events_dev  

MLP

'../logs/2019-02-22T08:52/private_events_dev/summary.csv'

6.5h

val_acc 
min 0.558559;
max 0.781818 

mean and 95% confidence level is 0.6686 0.0821

95% confidence interval is 0.5675 to 0.7435


